In [3]:
### IMPORT PACKAGES ###

import pandas as pd
import numpy as np

import datetime
import time
from sklearn.neighbors import KNeighborsClassifier

In [4]:
### IMPORT THE DATA ###
df = pd.read_csv('data/BPI_Challenge_2012-training.csv')

# select columns of interest
instance_column = "case concept:name"
event_column = "event concept:name"
timestamp_column = "event time:timestamp"
additionalInfo = [] # like 'event lifecycle:transition'

# putting the columns together and eliminating other columns
columns = [instance_column, event_column, timestamp_column]
columns.extend(additionalInfo)
dataset = df[columns]

# adding an index column
row_nr_column = 'row_nr'
dataset[row_nr_column] = dataset.index

# variables
timestampFormat = "%d-%m-%Y %H:%M:%S.%f"

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# checking the dataset

dataset.head()

,case concept:name,event concept:name,event time:timestamp,row_nr
0,173688,A_SUBMITTED,01-10-2011 00:38:44.546,0
1,173688,A_PARTLYSUBMITTED,01-10-2011 00:38:44.880,1
2,173688,A_PREACCEPTED,01-10-2011 00:39:37.906,2
3,173688,W_Completeren aanvraag,01-10-2011 00:39:38.875,3
4,173691,A_SUBMITTED,01-10-2011 08:08:58.256,4


In [6]:
# sorting by date

dataset.loc[:,timestamp_column] = pd.to_datetime(dataset.loc[:,timestamp_column], format=timestampFormat)
dataset = dataset.sort_values(by=timestamp_column)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [4]:
### STORAGE ###
# we want to keep track of things that happen within one instance (also called trace or case)

# instanceStorage stores: key=instance, values= [event_column, timestamp_column],
# to look up the previous event in this instance
instanceStorage = {}

# we also want to keep track of the predictionError
# errorStorage stores: key=unique integer, values= [previousEvent, previousTimestamp, predictionError, predictionErrorRatio]
errorStorage = {}


def updateStorage(case, event, timestamp, row_nr, predicted_event):
    # if this instance is already in the instanceStorage, we have a previous event
    if case in instanceStorage:

        # find the previous event
        previousEvent = instanceStorage[case]

        # time related variables
        thisTimestamp = pd.to_datetime(timestamp, format=timestampFormat)
        previousTimestamp = pd.to_datetime(instanceStorage[case][1], format=timestampFormat)
        timePassed = thisTimestamp - previousTimestamp
        
        if timePassed >= certain_time:
            something = True
        if timePassed >= scalar * regular_time_for_event
            something_else = True

        # event related variables
        this_event = event
        previous_event = instanceStorage[case][0]

        # since we have also saved the predicted time between the previous event and this event,
        # we can now also compute the difference
        if predicted_event != None:

            errorStorage[row_nr] = [previousEvent, previousTimestamp, predictionError, predictionErrorRatio]

    # otherwise we don't have a previous event
    else:
        previousEvent = None
        timePassed = None

    # key = instance_column (so which case we're looking at), and the values are the activity and the timestamp
    instanceStorage[case] = [event, timestamp]

    return previousEvent, timePassed


instanceStorage